<a href="https://colab.research.google.com/github/Knockjoy/browse-on-google-colab/blob/main/browser9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

In [30]:
#@title settings
port = "8000"#@param {type:"string"}
host = True #@param {type:"boolean"}

In [31]:
#@title shell
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg
Package: *
Pin: release a=eoan
Pin-Priority: 500
Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300
Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF
apt-get update
apt-get install chromium chromium-driver
pip install selenium
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.4.7.4
pip install flask

Executing: /tmp/apt-key-gpghome.rTA60Kg6rp/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.6BfZu3IVv9/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.DGbJg5twyK/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [91]:
#@title create browser.py
browser_py='''
from selenium_profiles.utils.installer import install_chromedriver
from selenium_profiles.driver import driver as mydriver
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By
from selenium_profiles.utils.colab_utils import display, showscreen, show_html 
class Brows():
  def startup(self):
    install_chromedriver()
    self.mydriver = mydriver()
    self.display = display()
    self.display.start_display()
    self.driver = self.mydriver.start(profiles.Windows(), uc_driver=False)  # or .Android    
  def access(self,url):
    print('access now')
    self.driver.get(url)
    self.save_screen('browser/static/screen.png')
    # showscreen(self.driver)
  def save_screen(self,path):
    self.driver.save_screenshot(path)
  def exit(self):
    self.driver.quit()
    self.display.stop_display()
  def search_ongoogle(self,word):
    self.access(f'https://google.com/search?q={word}')
'''.strip()


In [92]:
# from browser import browser
# chrome_browser = browser.Brows()
# chrome_browser.startup()
# chrome_browser.access("https://twitter.com")
# chrome_browser.exit()

In [93]:
project_name="browser"
from google.colab.output import eval_js
host = eval_js(f"google.colab.kernel.proxyPort({str(port)})")

In [112]:
#@title app.py
app_py_string = '''
import sys
import browser
from flask import Flask, redirect ,request,render_template,jsonify
import json
import requests

class ctrl_driver:
  def driver_run(self):
    self.chrome_browser = browser.Brows()
    self.chrome_browser.startup()
    self.chrome_browser.access("https://twitter.com")

  def driver_exit(self):
    self.chrome_browser.exit()

app = Flask(__name__)

@app.route("/")
def check():
    return render_template("index.html")

# @app.route("/output", methods=["POST"])
# def output():
#     try:
#         _A = int(request.json["_A"])
#         _B = int(request.json["_B"])
#         res = _A + _B
#         data = {"result":res, "_A":_A, "_B":_B, "msg":"%s + %s = %s"%(_A,_B,res)}
#         return jsonify(ResultSet=json.dumps(data)), 200
#     except Exception as err:
#         print(err)
#         data = {"error":"err"}
#         return jsonify(ResultSet=json.dumps(data)), 500

if __name__ == "__main__":
    import sys
    args = sys.argv
    dirive=ctrl_driver()
    dirive.driver_run()
    app.run(debug=True, port=args[1])
'''.strip()

In [113]:
#@title index.html
index_html_string = '''

<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>Document</title>
</head>
<body>
    <input type="text" value="0" name="_A" /> + <input type="text" value="0" name="_B" /> <button id="run">を計算する</button><br/>
    <h2>結果</h2>
    <script src="static/post.js"></script>
</body>
</html>

'''.strip()

In [114]:
#@title post.js
post_js_string='''

fetch("https://code.jquery.com/jquery-3.4.1.min.js").then(r=>{return r.text()}).then(t=>eval(t)).then(()=>{
	$=jQuery;
    document.getElementById("run").addEventListener("click",
        function(){
            values=[...document.querySelectorAll("input")].map(_=>_.value);
            $.ajax({
                url:"/output",
                type:"POST",
                data:JSON.stringify({"_A":values[0],"_B":values[1]}),
                contentType:"application/json"
            }).done(data=>{
                $(document.body).append(JSON.parse(data.ResultSet).msg,"<br/>");
            }).fail(data=>{
                $(document.body).append("error","<br/>")
            });
        },false);
});

'''.strip()


In [115]:
!rm -rf $project_name
!mkdir $project_name && mkdir $project_name/templates && mkdir $project_name/static
!echo -e '{app_py_string}' >> $project_name/app.py
!echo -e '{index_html_string}' >> $project_name/templates/index.html
!echo -e '{post_js_string}' >> $project_name/static/post.js
!echo -e "{browser_py}">> $project_name/browser.py

In [116]:
print('①が起動してから、↓のurlにアクセスする')
print(host)
#①
!python $project_name/app.py $port

①が起動してから、↓のurlにアクセスする
https://ybt4tj1860k-496ff2e9c6d22116-8000-colab.googleusercontent.com/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  chromium-chromedriver
0 upgraded, 1 newly installed, 0 to remove and 77 not upgraded.
Need to get 2,496 B of archives.
After this operation, 77.8 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 chromium-chromedriver amd64 1:85.0.4183.83-0ubuntu0.20.04.3 [2,496 B]
Fetched 2,496 B in 0s (10.8 kB/s)
(Reading database ... 128803 files and directories currently installed.)
Preparing to unpack .../chromium-chromedriver_1%3a85.0.4183.83-0ubuntu0.20.04.3_amd64.deb ...
Unpacking chromium-chromedriver (1:85.0.4183.83-0ubuntu0.20.04.3) ...
dpkg: error processing archive /var/cache/apt/archives/chromium-chromedriver_1%3a85.0.4183.83-0ubuntu0.20.04.3_amd64.deb (--unpack):
 trying to overwrite '/usr/bin/ch

In [117]:
!ls

'=2.2.4.7.4'   browser	 chromedriver_linux64.zip   sample_data   screen.png
